# Citation classifier excite


GIT repo: `git clone git@nopro.be:james/excite.git` 

Implementation of LSTM that can classify citation strings into regions denoting author, article title, journal, volume, year, pages, doi, notes and some other classes. There are 13 classes in total.

## Preparing Data regions

The following function takes citations that have been annotated and builds a mapping of character to classes. Since neural networks are also completely numerical constructs, we create an alphabet that maps numerical indices in a vector to letters.


In [1]:
import lxml.etree as ET
from collections import Counter

def prepare_data(citefile):

    #a list of all citations in training data
    citations = []
    #letters is essentially our alphabet used to map alphanumerical chars to integers
    letters = set()
    #this is a map of example classes found in the training data
    classes = Counter()

    with open(citefile) as f:

        for line in f.readlines():
            #add citation element to beginning and end of each example line so that we can parse into xml doc
            root = ET.XML("<citation>" + line.replace("&","&amp;") + "</citation>")
            cite = ""
            regions = []

            #iterate over child elements of our citation doc
            for el in root.iterchildren():
                classes[el.tag] = 1
                regions.append( (el.tag, len(cite)) )
                cite += el.text.replace("&amp;","&")

            #letters is a set so we just union against our new citation string to get unique chars used
            letters = letters.union( set(list(cite)) )
            citations.append( (cite, regions) )

    return citations, sorted(classes.keys()), sorted(letters)

We can use this function to build test and training sets. We can import data from the training sets attached

In [2]:
#extract training data from examples
citations, classes, alphabet = prepare_data("excite/data/citeseerx.tagged.txt")

# Find out how many classes there were in the training data as a sanity check
print ( str.format("Total classes found: {}",len(classes) ) )
print ("Classes: ", classes)
    


Total classes found: 13
Classes:  ['author', 'booktitle', 'date', 'editor', 'institution', 'journal', 'location', 'note', 'pages', 'publisher', 'tech', 'title', 'volume']


## Architecture of Network

Now that we have an idea of the number of classes we can start to plan network architecture.

### Network input

We are using a 5 character context window over our time series (which is essentially moving the context window along one character at a time. We need a context window function, $c(s,t)$ which given an input citation $s$ and an offset, $t$ creates a context window for network input, $x$

So you might expect the following:

In [3]:
#given a citation string that looks like this
citation = citations[0][0].strip()

print (citation)

x_t_1 = list(citation[0:5])

print ("x for t=1: ", x_t_1)

x_t_2 = list(citation[1:6])

print ("x for t=2: ", x_t_2)

Abbey, A. & Andrews, F. M.  Modeling the psychological determinants of life quality.  Social Indicators Research  16, 1,  1985.
x for t=1:  ['A', 'b', 'b', 'e', 'y']
x for t=2:  ['b', 'b', 'e', 'y', ',']


Now we want to make sure that the character we're interested in (which aligns with offset $t$ is at the centre of this matrix. The first character in this string is 'A' so we pad it with emptystring like so:

In [4]:
lpadded = 2 * [''] + list(citation) + 2 * ['']

x_t_1 = lpadded[0:5]

x_t_2 = lpadded[1:6]

print ("x for t=1: ", x_t_1)

print ("x for t=2: ", x_t_2)

x for t=1:  ['', '', 'A', 'b', 'b']
x for t=2:  ['', 'A', 'b', 'b', 'e']


The problem is that neural networks are completely numerical and strings must be encoded as numbers in order to be passed in. Therefore we use the alphabet collected above to map our x values to something more RNN friendly.

In [5]:
import numpy as np

def amap(letters):
    return np.array([ alphabet.index(x) if x in alphabet else -1 for x in letters])

print( "Encoded x for t=1", amap(x_t_1) )
print( "Encoded x for t=2", amap(x_t_2) )

Encoded x for t=1 [-1 -1 24 55 55]
Encoded x for t=2 [-1 24 55 55 58]


So we can wrap all this up in a function very easily now:

In [6]:
def cite_to_input(citestring, t):
    lpadded = 2 * [''] + list(citestring) + 2 * ['']
    cwin = lpadded[ 0 + t : 5 + t]
    return amap(cwin)

print(amap(x_t_1))
print (cite_to_input(citation, 0))

[-1 -1 24 55 55]
[-1 -1 24 55 55]


    
### Network output
    

The output will be a vector of values between 0,1 as wide as the number of classes. 

The aim is to use back propogation through time to make the input match with an output of all zeroes except the correct class in the vector space.

In [7]:
import theano
import theano.tensor as T


#we know the input context window is 5 
n_in = 5
#the output is the number of classes (currently 13)
n_out = len(classes)

#hidden units somewhere between input and output - we'll try 10 for now 
n_hidden = 10

# we set up the various layers of the network 

#the input layer is just a vector - yes it is 5 wide but we don't care about this yet
L_in = T.dvector('x_in')

# States is a vector of memory unit values
S_lstm = theano.shared(np.zeros(n_hidden))

# we want to store the previous state of the LSTM output for future calculations ( y(t-1))
y_lstm_t = theano.shared(np.zeros(n_hidden))

#output layer is a vector of values which will eventually be len(classes) wide
L_out = T.dvector('y_out')

# set up all the weights - these are matrices that weight values map connections between layers
# W[l,m] is the weight of connection from unit m to unit l

#weights for hidden layer internal connections 
W_hh = theano.shared(np.random.randn(n_hidden,n_hidden) * 0.001)
#weights for input to hidden connections
W_hi  = theano.shared( np.random.randn(n_hidden, n_in) * 0.001 )
#weights for hidden to out
W_oh  = theano.shared( np.random.randn(n_out, n_hidden ) * 0.001 )

#weight for input gate function 
W_in = theano.shared(value=np.random.randn() * 0.001, name='W_in')
#weight for forget gate function
W_phi = theano.shared(value=np.random.randn() * 0.001, name='W_phi')
#weight for output gate function
W_out = theano.shared(value=np.random.randn() * 0.001, name='W_out')




## Forward Propagation

This section is an implementation of Forward prop described in ['Learning Precise Timing with LSTM Recurrent Networks' - Gers et al. 2002](http://www.jmlr.org/papers/volume3/gers02a/gers02a.pdf)

Each time the sequence advances (or each time $t$ goes up by 1) we have to forward propagate the input $x$ through the network and calculate output. Here we define what that looks like.

### Input Gate

$$ z_{c^v_j}(t) = \sum_m w_{c^v_j m} y_m (t-1)  $$


In [8]:
z_lstm =  W_hi.dot(L_in) + W_hh.dot( y_lstm_t )

#compile a function to demo what this looks like
f = theano.function( [L_in], z_lstm)

print(x_t_1)
print(amap(x_t_1))
print( f( amap(x_t_1) ) )

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
[-0.00604327  0.03790514 -0.04192091  0.08768305  0.0034655  -0.05841105
  0.15559771 -0.00504544  0.0811      0.00613676]


$$ z_{in_j}(t)=\sum_m w_{in_j m} y_m (t-1)$$

$$y_{in_j}(t)= f_{in_j}(z_{in_j}(t))$$

In [9]:
y_in_lstm =  T.nnet.sigmoid( L_in.dot(W_in).sum() + y_lstm_t.dot( W_in ).sum() )

#compile a function to demo what this looks like
f = theano.function( [L_in], y_in_lstm )

print(x_t_1)
print(amap(x_t_1))
print( f( amap(x_t_1) ) )

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
0.5070567957478449


### Forget gate



$$ z_{\varphi_j}(t)=\sum_m w_{\varphi_j m} y_m (t-1)$$

$$y_{\varphi_j}(t)= f_{\varphi_j}(z_{\varphi_j}(t))$$

In [10]:
y_phi_lstm = T.nnet.sigmoid( L_in.dot(W_phi).sum() + y_lstm_t.dot( W_phi ).sum() )

#compile a function to demo what this looks like
f = theano.function([L_in], y_phi_lstm )

print(x_t_1)
print(amap(x_t_1))
print( f( amap(x_t_1) ) )

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
0.5225861167139391


### Cell state

If $ t \gt 0$ then it is calculated like so:

$$s_{c^v_j} (t) =  y_{\varphi j}(t)s_{c^v_j} (t-t) + y_{in_j}(t) g(z_{c^v_j}(t))  $$


In [11]:
s_lstm = ( y_phi_lstm * S_lstm) + (y_in_lstm * z_lstm )

f = theano.function([L_in],s_lstm)

print(x_t_1)
print(amap(x_t_1))
print(f( amap(x_t_1) ))

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
[-0.00306428  0.01922006 -0.02125628  0.04446028  0.00175721 -0.02961772
  0.07889688 -0.00255833  0.04112231  0.00311169]


### Output activation function

$$ z_{out_j}(t)=\sum_m w_{out_j m} y_m (t-1)$$

$$y_{out_j}(t)= f_{out_j}(z_{out_j}(t))$$

In [12]:
y_out_lstm =  T.nnet.sigmoid( L_in.dot(W_out).sum() + y_lstm_t.dot( W_out ).sum() )

#compile a function to demo what this looks like
f = theano.function( [L_in], y_out_lstm )

print(x_t_1)
print(amap(x_t_1))
print( f( amap(x_t_1) ) )

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
0.535537531526583


### LSTM Layer Output value

$$ y_{c^v_j}(t) =y_{out_j}(t) s_{c^v_j}(t) $$

In [13]:
y_lstm = y_out_lstm * s_lstm

f = theano.function([L_in],y_lstm)

print(x_t_1)
print(amap(x_t_1))
print(f( amap(x_t_1) ))

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
[-0.00164104  0.01029306 -0.01138354  0.02381015  0.00094105 -0.0158614
  0.04225224 -0.00137008  0.02202254  0.00166642]


## Network output

The final layer is a standard densely connected neural network layer len(classes) in size. y_k(t)$ denotes the final output of the network.

$$ y_k(t) = f_k(z_k(t))$$
$$ z_k(t) = \sum_m w_{km} y_m(t) $$

In [14]:
z_k = W_oh.dot(y_lstm)
y_k = T.nnet.softmax( z_k )

f = theano.function([L_in], y_k)

print(x_t_1)
print(amap(x_t_1))
print(f( amap(x_t_1) ))

['', '', 'A', 'b', 'b']
[-1 -1 24 55 55]
[[ 0.07691889  0.07692877  0.07692734  0.07692417  0.07692361  0.07692447
   0.07692004  0.07692868  0.07692535  0.07692062  0.07691622  0.07692303
   0.07691879]]


## Backwards Pass


### References

Implementation of Backwards Propagation through time (BPTT) - truncated gradients.

Implementation details described in 
  * ['Long Short Term Memory' Hochreiter & Schmidhuber 1997](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
  * ['Learning to Forget: Continual Prediction with LSTM' Gers et al. 1999](http://www.felixgers.de/papers/FgGates-ICANN99.pdf) 
  * ['Learning Precise Timing with LSTM Recurrent Networks' - Gers et al. 2002](http://www.jmlr.org/papers/volume3/gers02a/gers02a.pdf)
  
### Outputs and Output Gate
  
#### Defining our targets $t(t)$
First we define targets $t(t)$ which represent the correct answers.

Our input is a 5 wide vector of chars with the character we are interested in at its centre as demonstrated by the following where X is the letter we care about: [ \_ , \_, X \_, \_   ]. 

Our examples are divided up into boundaries so we just need to find the correct class for time $t$ (or rather offset $t$ if it helps to think that way).


In [15]:
def class_for_offset(regions, t):
    
    for cls, idx in sorted(regions, key=lambda x: x[1], reverse=True):
        if t >= idx:
            return cls
    else:
        return regions[-1][0]


citestring, regions = citations[0]

print (regions)

print( class_for_offset(regions, 12))
print( class_for_offset(regions, 30))
print( class_for_offset(regions, 90))
print( class_for_offset(regions, 115))
print( class_for_offset(regions, 123))


[('author', 0), ('title', 28), ('journal', 86), ('volume', 114), ('date', 122)]
author
title
journal
volume
date


Then its as simple as replacing the class name with an index to make 1 where everything else is zero



In [16]:
def output_vec(regions, t):
    cls = class_for_offset(regions,t)
    out = np.zeros(len(classes))
    out[classes.index(cls)] = 1
    return out


citestring, regions = citations[0]

print( output_vec(regions,12))

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


So now we have $t(t)$ we can start building BPTT.

#### Minimising error and gradient descent

Our error function is root mean squared error of our expected and actual outputs

$$E(t) = \frac{1}{2}\sum_ke_k(t)^2$$  
$$e_k(t) := t_k(t) - y_k(t)$$

In [17]:
t_k = T.dvector('t_k')
E_t = 0.5 * T.sqr((t_k - y_k)).sum()

f = theano.function([L_in, t_k], E_t)
y = theano.function([L_in], y_k)

citation, regions = citations[0]
input  = cite_to_input(citation,0)
target = output_vec(regions, 0)
print ("Input:", input)
print ("Desired Output:",target)
print ("Actual Output:", y(input))
print ("Error E(t)=", f(input, target))



Input: [-1 -1  0 24 55]
Desired Output: [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Actual Output: [[ 0.07691939  0.07692331  0.07692656  0.07692295  0.07692385  0.07692451
   0.07692307  0.07692718  0.07692553  0.07692164  0.07691796  0.07692391
   0.07692015]]
Error E(t)= 0.46154214912007907


We want to minimize $E(t)$ by doing gradient descent weight changes $\Delta w_{lm}$ using learning rate $\alpha$:

$$ \Delta w_{lm}(t) = \alpha \delta_k (t) y_m(t-1) $$

$$\delta_k (t) = f'_k(z_k(t))e_k(t)$$

$$ e_k(t) = t_k(t) - y_k(t) $$

Remembering that:
  * $\Delta w_{lm}(t)$ is the change in weight between current cell $l$ and its input cell $m$
  * $y^m(t-1)$ is feeder cell $m$'s previous input value (for $t-1$) and which we store in our program as `y_lstm_t`
  

In [21]:
e_k = t_k - y_k

learning_rate = 0.1

delta_k = T.grad(E_t,y_k) * e_k

delta_W_oh = learning_rate * T.grad(E_t, W_oh)

f = theano.function([L_in, t_k], W_oh - delta_W_oh) #theano.function([L_in,t_k], learning_rate * ( delta_k.reshape((1,13)).dot(y_lstm_t) ) ) 


#print ("W_oh: ", W_oh.get_value())
#print ("W_oh - delta_W_oh: ", f(input,target))

updates = []
updates.append((W_oh, W_oh - delta_W_oh))
updates.append( (y_lstm_t, y_lstm) )



### Output Gate

$$\delta_{out_j}(t) = f'_{out_j}(z_{out_j}(t)) \Big( \sum_{v=1}^{S_j} s_{c^v_j}(t) \sum_k W_{kc^v_j} \delta_k(t) \Big) $$

In [23]:

delta_W_out = T.grad(E_t, W_out)

f = theano.function([L_in, t_k], delta_W_out) #theano.function([L_in,t_k], learning_rate * ( delta_k.reshape((1,13)).dot(y_lstm_t) ) ) 

print ("W_out:", W_out.get_value())
print ("W_oh - delta_W_out: ", f(input,target))

W_out: 0.00107871378158
W_oh - delta_W_out:  0.00013607913091728883


## Single step

Now that we have weights defined, we can define what it means to do a single step given time, $t$. This involves forward propogating new (and recycled inputs) and then backpropogating error signals.

Here we define a python function, $step$ that carries out these processes


In [203]:

def step(x,y):
    pass